# Chrome Remote Desktop attempt + fallback to noVNC

This notebook will:
1. Attempt to install Chrome Remote Desktop (CRD). CRD requires a headless authorization step at https://remotedesktop.google.com/headless and may fail in Colab. Expect fragility.
2. If CRD isn't registered or fails, the notebook will automatically fall back to starting an in-container Xfce desktop exposed via x11vnc + noVNC and public URLs via cloudflared.

Both paths include code-server (VS Code) running on localhost:8080. Cloudflared is used to publish public URLs (not ngrok).

Important security notes: change the default passwords, and do not expose sensitive data. Colab sessions are ephemeral and may be killed at any time.

In [ ]:
%%bash
set -e
echo "Installing desktop components, code-server, noVNC/websockify, and cloudflared (may take a few minutes)..."
apt-get update -qq
DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
  xfce4 xfce4-terminal x11vnc xvfb wget git python3-pip ca-certificates curl gnupg2 unzip \
  >/dev/null

# Install noVNC and websockify
rm -rf /opt/noVNC /opt/websockify
git clone https://github.com/novnc/noVNC.git /opt/noVNC
git clone https://github.com/novnc/websockify.git /opt/websockify

# Install cloudflared (latest release binary)
wget -q -O /usr/local/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
chmod +x /usr/local/bin/cloudflared

# Install code-server (VS Code in browser)
curl -fsSL https://code-server.dev/install.sh | sh

echo "Install finished. Next: try to install Chrome Remote Desktop and Google Chrome (this step is likely to be fragile in Colab)."
echo "If CRD registration fails, continue to the fallback cells (noVNC + cloudflared)."


## Attempt Chrome Remote Desktop (manual step required)

If you want to try CRD, follow these steps:
1. Open https://remotedesktop.google.com/headless in your browser and sign in with the Google account you want to use.
2. On that page select **Set up remote access** → **Begin** → **Generate code** (the site will give you a long command that looks like `bash -c "... --pin=XXXX ..."`).
3. Copy the entire command the page shows (the "registration command").
4. Paste that command into the next cell (replacing the placeholder) and run it.

Note: If this registration step fails (very possible in Colab), skip it and use the fallback cells below.

In [ ]:
%%bash
# ----- PASTE THE REGISTRATION COMMAND BELOW: -----
# Example (DO NOT RUN the example; replace it with the real command you get from the headless page):
# bash -c "DISPLAY= :0 /opt/google/chrome-remote-desktop/start-host --code="XXXXXXXX" --redirect-url=https://remotedesktop.google.com/_/oauthredirect --name=$(hostname) --pin=123456"

# If you have a real registration command, paste it here (uncomment) and run this cell.
echo "No CRD registration command provided in this cell. Skipping CRD registration."


In [ ]:
%%bash
set -e
echo "--- Starting fallback: Xvfb + Xfce + x11vnc + noVNC + code-server ---"

# User-settable password for code-server and (optionally) VNC. Change before exposing publicly.
PASSWORD=${PASSWORD:-"colab"}

export DISPLAY=:1
Xvfb $DISPLAY -screen 0 1280x800x24 >/tmp/xvfb.log 2>&1 &
sleep 1
nohup startxfce4 >/tmp/xfce.log 2>&1 &
sleep 2

x11vnc -display $DISPLAY -nopw -forever -shared -rfbport 5901 >/tmp/x11vnc.log 2>&1 &
sleep 1

# Start websockify (noVNC web UI) on 6080
/opt/websockify/run 6080 localhost:5901 --web=/opt/noVNC >/tmp/novnc.log 2>&1 &
sleep 1

mkdir -p ~/.config/code-server
cat > ~/.config/code-server/config.yaml <<EOF
bind-addr: 127.0.0.1:8080
auth: password
password: $PASSWORD
cert: false
EOF

nohup code-server --bind-addr 127.0.0.1:8080 --auth password >/tmp/code-server.log 2>&1 &

echo "Fallback services started. code-server on 127.0.0.1:8080 (password: $PASSWORD). noVNC available on 127.0.0.1:6080."


In [ ]:
%%bash
set -e
echo "Starting cloudflared tunnels for code-server (8080) and noVNC (6080). Logs will show public URLs when available."
nohup cloudflared tunnel --url http://localhost:8080 > /tmp/cloudflared_8080.log 2>&1 &
nohup cloudflared tunnel --url http://localhost:6080 > /tmp/cloudflared_6080.log 2>&1 &
sleep 5
echo "-- cloudflared log (8080) --"
tail -n +1 /tmp/cloudflared_8080.log || true
echo "\n-- cloudflared log (6080) --"
tail -n +1 /tmp/cloudflared_6080.log || true

echo "If you see lines that include a .trycloudflare.com URL, those are the public endpoints. If not, give it a few seconds and re-run the tail command."


## How to connect

- Open the public URL printed in the cloudflared logs for code-server. Enter the password you set (env var `PASSWORD` or default `colab`).
- Open the public URL printed for noVNC to view the Xfce desktop in your browser.

If you successfully completed the CRD registration step and see your machine listed at https://remotedesktop.google.com, you can attempt to connect via the Chrome Remote Desktop client instead of noVNC. But again: CRD in Colab is fragile and may not work reliably.

If you want, I can now also produce a single-file downloadable notebook (.ipynb) or paste-ready raw JSON for these two notebooks. You can also edit the `PASSWORD` variable at the top of the fallback cell before running to change access credentials.